In [ ]:
"""
Sliding Window:
1. maxProfit: Find max profit by buying/selling stock once.
2. longestSubString: Find longest substring without repeating characters.
3. minimumWindowSubString: Find smallest substring in s containing all chars of t.
4. characterReplacement: Longest substring with uniform characters after k replacements.
5. findAnagrams: Find all start indices of anagrams of p in s.
6. maxSlidingWindow: Find max in every window of size k.
7. maxFrequency: Max frequency achievable after k increments.
8. minFlip: Minimum flips to make binary string alternating.
9. checkInclusion: Check if s1's permutation is a substring of s2.
"""

class Solution:
    # 1. Best Time to Buy and Sell Stock (LeetCode 121)
    def maxProfit(self, prices):
        '''
        Find the maximum profit by buying and selling a stock once.
        '''
        maxProf = 0
        l, r = 0, 1

        while r < len(prices):
            # profitabel
            if prices[l] < prices[r]:
                profit = prices[r] - prices[l]
                maxProf = max(profit, maxProf)
            # not profitable and update the left pointer
            else:
                l = r
            r += 1

        return maxProf

    # Example:
    # Input: [7, 1, 5, 3, 6, 4]
    # Output: 5 -> 6 - 1


    # 2. Longest Substring Without Repeating Characters (LeetCode 3)
    def longestSubString(self, s):
        '''
        Find the longest substring without repeating characters.
        '''
        subString = set()
        result = 0
        l, r = 0, 0

        while r < len(s):
            # duplicate. start over
            while s[r] in subString:
                subString.remove(s[l])
                l += 1

            # no duplicate. extend substring
            subString.add(s[r])
            result = max(result, r - l + 1)
            r += 1

        return result

    # Example:
    # Input: "abcabcbb"
    # Output: 3


    # 3. Minimum Window Substring (LeetCode 76)
    def minimumWindowSubString(self, s, t):
        '''
        Find the smallest substring in `s` containing all characters of `t`.
        '''
        # 1. dictionary: char:+counters
        tDic, windows = {}, {}
        for char in t:
            tDic[char] = 1 + tDic.get(char, 0)

        # 2. Match counter
        need, have = len(tDic), 0
        # 3. Sliding Window
        windows_length, windows_res = float("infinity"), [-1, -1]
        l = 0
        for r in range(len(s)):
            # 1. Extend dictionary
            windows[s[r]] = 1 + windows.get(s[r], 0)

            # 2. Extend Match counter
            if s[r] in tDic and tDic[s[r]] == windows[s[r]]:
                have += 1

            # 3. Record sliding window
            while need == have:
                if r - l + 1 < windows_length:
                    windows_res = [l, r]
                    windows_length = r - l + 1

                # 1. Remove left dictionary and shift 
                windows[s[l]] -= 1
                # 2. Match counter
                if s[l] in tDic and tDic[s[l]] > windows[s[l]]:
                    have -= 1
                l += 1

        l, r = windows_res
        return s[l:r + 1] if windows_length != float("infinity") else ""

    # Example: Minimum Window Substring
    # Input: s = "ADOBECODEBANC", t = "ABC"
    # Output: "BANC"


    # 4. Longest Repeating Character Replacement (LeetCode 424)
    def characterReplacement(self, s, k):
        '''
        Find the longest substring you can make uniform with `k` replacements.
        
        a        b       a        b       a
        maxCount replace maxCount replace maxCount
        window = 5
        window - maxCount = 5 - 3 = 2 replacement = k

        reduce the counter for larger than k
        '''
        count = {}
        res = 0
        l = 0
        maxCount = 0
        for r in range(len(s)):
            # 1. add to dictionary
            count[s[r]] = 1 + count.get(s[r], 0)
            maxCount = max(maxCount, count[s[r]])

            # 2. reduce counter in dictionary and shift
            while (r - l + 1) - maxCount > k:
                count[s[l]] -= 1
                l += 1

            res = max(res, r - l + 1)

        return res

    # Example:
    # Input: s = "AABABBA", k = 1
    # Output: 4


    # 5. Find All Anagrams in a String (LeetCode 438)
    def findAnagrams(self, s, p):
        '''
        Find all start indices of p's anagrams in s.
        '''
        if len(p) > len(s): return []

        # 1. add to dictionary
        sCount, pCount = {}, {}
        for i in range(len(p)):
            pCount[p[i]] = 1 + pCount.get(p[i], 0)
            sCount[s[i]] = 1 + sCount.get(s[i], 0)
        res = [0] if pCount == sCount else []

        l = 0
        for r in range(len(p), len(s)):
            sCount[s[r]] = 1 + sCount.get(s[r], 0)
            # 2. reduce and remove from dictionary and shift
            # keep window size the same
            sCount[s[l]] -= 1
            if sCount[s[l]] == 0:
                sCount.pop(s[l])
            l += 1

            # 3. Find a match
            if sCount == pCount:
                res.append(l)

        return res

    # Example:
    # Input: s = "cbaebabacd", p = "abc"
    # Output: [0, 6]


    # 6. Sliding Window Maximum (LeetCode 239)
    def maxSlidingWindow(self, nums, k):
        '''
        Find the maximum in every sliding window of size k.
        '''
        output = []
        queue = []
        l, r = 0, 0
        while r < len(nums):
            # 1. Deque = top of stack
            # Decreasing Monotonic
            # Remove top of deque and add 
            while queue and nums[r] > nums[queue[-1]]:
                queue.pop()
            queue.append(r)

            # 2. Remove from front 
            if l > queue[0]:
                queue.pop(0)

            # add front of deque, which is max
            if r + 1 >= k:
                output.append(nums[queue[0]])
                l += 1
            r += 1
        return output

    # Example:
    # Input: nums = [1,3,-1,-3,5,3,6,7], k = 3
    # Output: [3,3,5,5,6,7]


    # 7. Frequency of Most Frequent Element (LeetCode 1838)
    def maxFrequency(self, nums, k):
        '''
        Find the max frequency of elements after at most `k` increments.
        '''
        nums.sort()
        l, r = 0, 0
        res, total = 0, 0

        while r < len(nums):
            '''
            # achievable sum
            total += nums[right] 
            # required sum
            window_sum = nums[right] * window_length
            '''
            # Add sums
            total += nums[r]
            # Reduce and shift
            while nums[r] * (r - l + 1) > total + k:
                total -= nums[l]
                l += 1
            res = max(res, r - l + 1)
            r += 1

        return res

    # Example:
    # Input: nums = [1,2,4], k = 5
    # Output: 3


    # 8. Minimum Flips to Make Binary Alternating (LeetCode 1888)
    def minFlip(self, s):
        '''
        Find the minimum flips to make binary string alternating.
        '''
        '''
        "1100"-> rotational/conditional:  -> s + s
        "1100 1100"
        Alternate:
        "1010 1010"
        "0101 0101"

        difference window:
        diff1 += 1+1
        diff2 += 1+1
        '''
        n = len(s)
        s = s + s
        alt1, alt2 = "", ""
        l, r = 0, 0
        diff1, diff2 = 0, 0
        res = len(s)

        # create alternates
        for i in range(len(s)):
            alt1 += "0" if i % 2 else "1"
            alt2 += "1" if i % 2 else "0"

        while r < len(s):
            # Check Matches and add to counters
            if s[r] != alt1[r]:
                diff1 += 1
            if s[r] != alt2[r]:
                diff2 += 1

            # Undo and decreament from differences and shift
            if (r - l + 1) > n:
                if s[l] != alt1[l]:
                    diff1 -= 1
                if s[l] != alt2[l]:
                    diff2 -= 1
                l += 1

            # Answer
            if (r - l + 1) == n:
                res = min(res, diff1, diff2)

            r += 1
        return res

    # Example:
    # Input: s = "111000"
    # Output: 2


    # 9. Permutation in String (LeetCode 567)
    # same as anagram
    def checkInclusion(self, s1, s2):
        '''
        Check if s1's permutation is a substring of s2.
        '''
        frequency_s1, frequency_s2 = [0] * 26, [0] * 26

        for char in s1:
            frequency_s1[ord(char) - ord('a')] += 1

        for right in range(len(s2)):
            # Add to right
            frequency_s2[ord(s2[right]) - ord('a')] += 1

            # Reduce from left
            if right >= len(s1):
                left = right - len(s1)
                frequency_s2[ord(s2[left]) - ord('a')] -= 1

            # Check match
            if frequency_s2 == frequency_s1:
                return True

        return False

    # Example:
    # Input: s1 = "ab", s2 = "eidbaooo"
    # Output
